<a href="https://colab.research.google.com/github/AntonioWh1te/The-Journey/blob/main/Phase_2_1_Financial_Modeling_Prep_(FMP)_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Understanding the Task
* Goal: Fetch the closing prices of AAPL, NVDA, and TSLA over the past year, then plot them.

* Extensions
  1. Changing data frequency to daily or hourly
  2. Exploring different plot types (line, bar, candlestick).
  3. Possibly creating animations.

**Key Idea:** We'll use FMP's historical endpoints to retrieve data, parse it into Pandas, and visualize it with Matplotlib.

# Step 1: Explore the FMP API (Application Programming Interface)



In [ ]:
# For daily historical data:

url = "https://financialmodelingprep.com/api/v3"

# Depdning on the data frequency (daily or intraday), we append the correct endpoint (historical-price-full), (historical-chart/1hour)
